# Comparison between original model and model with delay

Acapacidade média de atendimento por passo em cada um dos modelos comparados deve ser a mesma. Para isso, o número máxmio de servidores no caso com atraso deve ser igual a $S_d$

$$S_d = t_m \cdot S_o$$

onde $t_m$ é o tempo médio de serviço (1+atraso), $S_o$ é o número de servidores no modelo original sem atrasos.

Para um atraso máximo no serviço de 3 passos ($t_{serM} = 3$), com atrasos distribuidos de maneira uniforme, o $t_m$ seria igual a 3. Para que a comparação seja válida, é necessário que $t_m$ seja um inteiro. Para que isso ocorra, $t_serM$ deve ser ímpar.

$$t_m = \frac{((t_{serM}+1)(t_{serM}+2)/2)-1}{t_{serM}} = \frac{2+3+4}{3} = 3 $$

In [1]:
using JuMP
using HiGHS
using Ipopt
using Juniper

include("utils.jl") # demand_generator_mat, printTable, plotData

plotData (generic function with 1 method)

### Comon Parametrers

In [2]:
# Objective function Costs
c_z = 1.5        # weight of served clients
c_ser = 2      # weight of server cost  
c_L = 1

# Objective function
lin_fobj(S, dr, Cin, phi, Z, L) = c_ser*sum(S) - c_z*sum(Z) +c_L*sum(L)

# Parameters
N = 10            # numner of examples
horiz = 20        # total horizon

# bounds
XM = 5            # max queue length 
YM = 10           # max buffer legnth before dropping calls
phiM = 9          # max adimission to queue

struct bounds
    XM
    YM
    phiM
    serM  # different for each model
    tserM # only defined for the model with delay
end

# iniital conditions
X0 = 4
Y0 = 6
L0 = 0
Z0 = 0

struct initial_conditions
    X0
    Y0
    L0
    Z0
end
ic = initial_conditions(X0, Y0, L0, Z0)

createDemands = true
if createDemands
    d_mat = demand_generator_mat(1, horiz, 14, "uniform", 1)
    a_mat = demand_generator_mat(1, horiz, 2,"uniform",0.5)
else
    d_fn = "..//CC_simple//d_mat_Thu_22_May_2025_19_39_50.txt";
    a_fn = "..//CC_simple//a_mat_Thu_22_May_2025_19_39_50.txt";
    d_mat = DelimitedFiles.readdlm(d_fn);
    a_mat = DelimitedFiles.readdlm(a_fn);
    
end

d = d_mat[1:horiz, 1];  # demand for incoming calls
a = a_mat[1:horiz, 1];  # abandonment for calls

struct result
    id
    X
    Y
    Z
    L
    n
    Q
    dr
    phi
    Cin
    Cout
    S
    Sl
    Sa
    Sst
    Sc
    Sin
    Saux
    b_opt
    J
    c_ser
    c_blr
end
   

### Parameters original model

In [3]:
nl = false # true for non linear obj func and false for linear obj func

# costs for the non linear obj func
c_blro = 140
c_sero = 1

serMo = 5  # number of servers

bdso = bounds(XM, YM, phiM, serMo, 0);

### Original model results

In [4]:
include("cc_os_original.jl")
optimal, X, Y, Z, L, n, Q, dr, phi, S, b_opt, J = cc_os_original(ic, bdso, c_sero, c_blro, d, a, nl, lin_fobj);
result_os_ori = result("os_ori", X, Y, Z, L, n, Q, dr, phi, nothing, nothing, S, nothing, nothing, nothing, nothing, nothing, nothing, b_opt, J, c_sero, c_blro)

print( "\n \n")
println("Balking rate: ", sum(L)/sum(L+Z))
println("Total number of servers used: ", sum(S))
print("c_ser*S + c_blr*B = ", c_sero*sum(S) + c_blro*sum(L)/sum(L+Z))

ok
 
Balking rate: 0.4411764705882353
Total number of servers used: 82.0
c_ser*S + c_blr*B = 143.76470588235293

In [5]:
include("cc_om_original.jl")
optimal, X, Y, Z, L, n, Q, dr, phi, S, b_opt, J = cc_om_original(ic, bdso, c_sero, c_blro, d, a, nl, lin_fobj);
result_om_ori = result("om_ori", X, Y, Z, L, n, Q, dr, phi, nothing, nothing, S, nothing, nothing, nothing, nothing, nothing, nothing, b_opt, J, c_sero, c_blro)

print( "\n \n")
println("Balking rate: ", sum(L)/sum(L+Z))
println("Total number of servers used: ", sum(S))
print("c_ser*S + c_blr*B = ", c_sero*sum(S) + c_blro*sum(L)/sum(L+Z))

ok
 
Balking rate: 0.3565270931998317
Total number of servers used: 95.0
c_ser*S + c_blr*B = 144.91379304797644

### Parameters model with delay

In [6]:
tserM = 3         # max delay in service time
tm = (((tserM+1)*(tserM+2)/2)-1)/tserM # average service time

c_serd = c_sero/tm
c_blrd = c_blro

serMd = serMo*tm         # number of servers
serMd = Int(serMd)
bdsd = bounds(XM, YM, phiM, serMd, tserM)

rnd_tser = true
df_input = df_input_generator(horiz, serMd, tserM, rnd_tser);

### Delay model results

In [7]:
include("cc_os_delay.jl")
optimal, X, Y, Z, L, n, Q, dr, phi, Cin, Cout, S, Sl, Sa, Sst, Sc, Sin, Saux, b_opt, J = cc_os_delay(ic, bdsd, a, d, df_input, lin_fobj);
result_os_del = result("os_del", X, Y, Z, L, n, Q, dr, phi, Cin, Cout, S, Sl, Sa, Sst, Sc, Sin, Saux, b_opt, J, c_serd, c_blrd)

print( "\n \n")
println("Balking rate: ", sum(L)/sum(L+Z))
println("Total number of servers used: ", sum(S)/tm)
print("c_ser*S + c_blr*B = ", c_serd*sum(S) + c_blrd*sum(L)/sum(L+Z))


 
Balking rate: 0.4311639549436796
Total number of servers used: 83.66666666666667
c_ser*S + c_blr*B = 144.0296203587818

In [8]:
include("cc_om_delay.jl")
optimal, X, Y, Z, L, n, Q, dr, phi, Cin, Cout, S, Sl, Sa, Sst, Sc, Sin, Saux, b_opt, J = cc_om_delay(ic, bdsd, a, d, df_input, lin_fobj);
result_om_del = result("om_del", X, Y, Z, L, n, Q, dr, phi, Cin, Cout, S, Sl, Sa, Sst, Sc, Sin, Saux, b_opt, J, c_serd, c_blrd)

print( "\n \n")
println("Balking rate: ", sum(L)/sum(L+Z))
println("Total number of servers used: ", sum(S)/tm)
print("c_ser*S + c_blr*B = ", c_serd*sum(S) + c_blrd*sum(L)/sum(L+Z))

ok
 
Balking rate: 0.3139178418148375
Total number of servers used: 88.0
c_ser*S + c_blr*B = 131.94849785407726

## Results comparison

In [9]:
results_list = [result_os_ori, result_om_ori, result_os_del, result_om_del]

for res in results_list
    br = sum(res.L)/sum(res.L+res.Z)
    total_clients = res.Z[horiz+1]
    cost = res.c_ser*sum(res.S) + res.c_blr*sum(res.L)/sum(res.L+res.Z)

    println(res.id)
    data = hcat(br, total_clients, cost)
    header = (["br", "total clients", "cost"]);
    printTable(data, header)
end



os_ori
Results
┌───────┬───────────────┬─────────┐
│    br │ total clients │    cost │
├───────┼───────────────┼─────────┤
│ 0.441 │        82.000 │ 143.765 │
└───────┴───────────────┴─────────┘
om_ori
Results
┌───────┬───────────────┬─────────┐
│    br │ total clients │    cost │
├───────┼───────────────┼─────────┤
│ 0.357 │        95.000 │ 144.914 │
└───────┴───────────────┴─────────┘
os_del
Results
┌───────┬───────────────┬─────────┐
│    br │ total clients │    cost │
├───────┼───────────────┼─────────┤
│ 0.431 │        84.000 │ 144.030 │
└───────┴───────────────┴─────────┘
om_del
Results
┌───────┬───────────────┬─────────┐
│    br │ total clients │    cost │
├───────┼───────────────┼─────────┤
│ 0.314 │        98.000 │ 131.948 │
└───────┴───────────────┴─────────┘
